In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [2]:
data = pd.read_csv('Data/Support_tickets.csv')
data.head()

,ticket_id,day_of_week,day_of_week_num,company_id,company_size,company_size_cat,industry,industry_cat,customer_tier,customer_tier_cat,...,downtime_min,payment_impact_flag,security_incident_flag,data_loss_flag,has_runbook,customer_sentiment,customer_sentiment_cat,description_length,priority,priority_cat
0,1000000000,Wed,3,100015,Small,1,media,7,Basic,1,...,6,0,0,0,0,neutral,2,227,low,1
1,1000000001,Sat,6,100023,Small,1,healthcare,5,Basic,1,...,2,0,0,0,0,neutral,2,461,low,1
2,1000000002,Mon,1,100012,Small,1,gaming,4,Basic,1,...,0,0,0,0,1,positive,3,306,low,1
3,1000000003,Wed,3,100003,Small,1,media,7,Plus,2,...,16,0,0,0,1,neutral,2,363,medium,2
4,1000000004,Mon,1,100019,Small,1,ecommerce,2,Plus,2,...,6,0,0,0,0,neutral,2,442,low,1


In [3]:
columns = ['ticket_id', 'day_of_week_num', 'company_id', 'company_size_cat', 'industry_cat', 'customer_tier_cat',
           'region_cat','past_30d_tickets', 'past_90d_incidents', 'priority_cat', 'customers_affected','downtime_min',
           'error_rate_pct', 'payment_impact_flag', 'security_incident_flag', 'data_loss_flag', 'has_runbook',
           'reported_by_role_cat','booking_channel_cat', 'customer_sentiment_cat', 'description_length']

In [4]:
filtered_data = data[columns]
filtered_data.head()

,ticket_id,day_of_week_num,company_id,company_size_cat,industry_cat,customer_tier_cat,region_cat,past_30d_tickets,past_90d_incidents,priority_cat,...,downtime_min,error_rate_pct,payment_impact_flag,security_incident_flag,data_loss_flag,has_runbook,reported_by_role_cat,booking_channel_cat,customer_sentiment_cat,description_length
0,1000000000,3,100015,1,7,1,3,2,0,1,...,6,5.451201,0,0,0,0,1,1,2,227
1,1000000001,6,100023,1,5,1,1,2,3,1,...,2,2.495538,0,0,0,0,3,3,2,461
2,1000000002,1,100012,1,4,1,3,3,1,1,...,0,3.328402,0,0,0,1,2,3,3,306
3,1000000003,3,100003,1,7,2,1,2,2,2,...,16,2.931906,0,0,0,1,4,3,2,363
4,1000000004,1,100019,1,2,2,1,5,1,1,...,6,3.255222,0,0,0,0,1,1,2,442


In [5]:
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ticket_id               50000 non-null  int64  
 1   day_of_week_num         50000 non-null  int64  
 2   company_id              50000 non-null  int64  
 3   company_size_cat        50000 non-null  int64  
 4   industry_cat            50000 non-null  int64  
 5   customer_tier_cat       50000 non-null  int64  
 6   region_cat              50000 non-null  int64  
 7   past_30d_tickets        50000 non-null  int64  
 8   past_90d_incidents      50000 non-null  int64  
 9   priority_cat            50000 non-null  int64  
 10  customers_affected      50000 non-null  int64  
 11  downtime_min            50000 non-null  int64  
 12  error_rate_pct          50000 non-null  float64
 13  payment_impact_flag     50000 non-null  int64  
 14  security_incident_flag  50000 non-null

In [6]:
def percentile_calc(data, column, x):
    return data[column].quantile(x)

In [7]:
thresholds = {
    'customers_p90': filtered_data['customers_affected'].quantile(0.90),
    'error_rate_p90': filtered_data['error_rate_pct'].quantile(0.90),
    'downtime_p95': filtered_data['downtime_min'].quantile(0.95),
    'downtime_p90': filtered_data['downtime_min'].quantile(0.90),
    'downtime_p75': filtered_data['downtime_min'].quantile(0.75),
}

In [8]:
print(thresholds['downtime_p95'], thresholds['downtime_p90'], thresholds['downtime_p75'])

46.0 34.0 17.0


In [9]:
def sla_tier_assignment(data, thresholds):
    if (
        data['priority_cat'] == 3 and
        (
            data['security_incident_flag'] == 1 or
            data['data_loss_flag'] == 1 or
            data['payment_impact_flag'] == 1
        )
    ):
        return 'SLA-0'

    elif (
        data['customer_tier_cat'] == 3 and
        data['downtime_min'] >= thresholds['downtime_p95']
    ):
        return 'SLA-0'

    elif (
        data['priority_cat'] == 3 or
        data['customers_affected'] >= thresholds['customers_p90'] or
        data['error_rate_pct'] >= thresholds['error_rate_p90'] or
        data['customer_tier_cat'] == 3 or
        data['downtime_min'] >= thresholds['downtime_p90']
    ):
        return 'SLA-1'

    elif (
        data['priority_cat'] == 2 or
        data['downtime_min'] >= thresholds['downtime_p75']
    ):
        return 'SLA-2'

    else:
        return 'SLA-3'


In [10]:
filtered_data['SLA-Tier'] = filtered_data.apply(
    sla_tier_assignment,
    axis=1, thresholds = thresholds)


C:\Users\VISHNU\AppData\Local\Temp\ipykernel_15420\3072785439.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['SLA-Tier'] = filtered_data.apply(


In [22]:
def sla_pressure_calc(row):
    if row['SLA-Tier'] == "SLA-0":
        return 1.0
    elif row['SLA-Tier'] == "SLA-1":
        return 1/4
    elif row['SLA-Tier'] == "SLA-2":
        return 1/8
    elif row['SLA-Tier'] == "SLA-3":
        return 1/12
    else:
        return 0


In [12]:
scaler = MinMaxScaler()
filtered_data['n_customers_affected'] = scaler.fit_transform(filtered_data[['customers_affected']])
filtered_data['n_error_rate_pct'] = scaler.fit_transform(filtered_data[['error_rate_pct']])
filtered_data['recent_ticket_score'] = filtered_data['past_30d_tickets'] / max(filtered_data['past_30d_tickets'])
filtered_data['recent_incident_score'] = filtered_data['past_90d_incidents'] / max(filtered_data['past_90d_incidents'])

C:\Users\VISHNU\AppData\Local\Temp\ipykernel_15420\2025979368.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['n_customers_affected'] = scaler.fit_transform(filtered_data[['customers_affected']])
C:\Users\VISHNU\AppData\Local\Temp\ipykernel_15420\2025979368.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['n_error_rate_pct'] = scaler.fit_transform(filtered_data[['error_rate_pct']])
C:\Users\VISHNU\AppData\Local\Temp\ipykernel_15420\2025979368.py:4: SettingWithCopyWarning: 

In [13]:
def downtime_severiaty(row):
    if row['downtime_min'] <= thresholds['downtime_p75']:
        return 0.3
    elif row['downtime_min'] <=thresholds['downtime_p90'] and row['downtime_min'] >=thresholds['downtime_p75']:
        return 0.2
    elif row['downtime_min'] >=thresholds['downtime_p90']:
        return 0.3

In [14]:
def incident_flag(row):
    if  row['security_incident_flag'] == 1 or row['data_loss_flag'] == 1 or row['payment_impact_flag'] == 1:
        return 1
    else:
        return 0

In [15]:
def impact_scorer(row):
    downtime_factor = downtime_severiaty(row)
    incident_factor = incident_flag(row)
    customer_factor = row['n_customers_affected']
    error_factor = row['n_error_rate_pct']

    impact_score = (downtime_factor) + (customer_factor * 0.25) + (error_factor * 0.15) + (incident_factor * 0.3)
    return impact_score


In [18]:
def customer_weight(row):
    if row['customer_tier_cat'] == 3:
        return 1.5
    elif row['customer_tier_cat'] == 2:
        return 1.2
    elif row['customer_tier_cat'] == 1:
        return 1

In [19]:
def instability_weight(row):
    instability_factor = (row['recent_ticket_score']*0.6) * (0.4 * row['recent_incident_score'])
    return 1 + (0.5)*instability_factor

In [20]:
def SLA_Risk_Score(row):
    SLA_risk_score = customer_weight(row) * instability_weight(row) * impact_scorer(row) * sla_pressure_calc(row)
    return SLA_risk_score

In [23]:
filtered_data['SLA_Risk_Score'] = filtered_data.apply(SLA_Risk_Score, axis=1)

C:\Users\VISHNU\AppData\Local\Temp\ipykernel_15420\1045933887.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['SLA_Risk_Score'] = filtered_data.apply(SLA_Risk_Score, axis=1)


In [36]:
# filtered_data.head()

In [27]:
(filtered_data['SLA_Risk_Score'].max())

np.float64(1.1151970938103335)

In [28]:
filtered_data.to_csv("sla_risk.csv")

In [35]:
# plt.hist(filtered_data['SLA_Risk_Score'], bins = 100)
# plt.xlim(0.8, 1)
# plt.ylim(0,100)